In [ ]:
%matplotlib widget

In [ ]:
import matplotlib

import numpy as np
from pathlib import Path
import flammkuchen as fl
from tifffile import imread
import matplotlib.pyplot as plt 
from fimpylab.core.lightsheet_experiment import LightsheetExperiment
from bouterin.plots.stimulus_log_plot import get_paint_function

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr
from scipy.signal import detrend 

from motions.utilities import stim_vel_dir_dataframe, quantize_directions
import tifffile as tiff
from scipy.signal import argrelextrema
from scipy.signal import find_peaks

In [ ]:
master_path =  Path(r"Z:\Hagar and Ot\E0040\v10\LS")
fish_list = list(master_path.glob("*f*"))
path = fish_list[4]
print(path)

In [ ]:
traces = fl.load(path / "filtered_traces.h5", "/detr")
sens_corr = fl.load(path / "sensory_regressors.h5", "/regressors")

In [ ]:
n_t, n_neurons = np.shape(traces)
fs = 2

In [ ]:
mean_traces = np.nanmean(traces.T, axis=0)
mean_regs = np.nanmean(sens_corr.values, axis=0)

traces_std = np.nanstd(traces.T, axis=0)
regs_std = np.nanstd(sens_corr.values, axis=0)


# Normalize the data
traces_normalized = (traces.T - mean_traces) / traces_std
regs_normalized = (sens_corr.values - mean_regs) / regs_std
    
dot_product = np.dot(traces_normalized, regs_normalized)
reg_corr_values = dot_product / (n_t - 1)

In [ ]:
color_list = ['k', 'k', 'k', 'k', 'k', 'k', 'k', 'k']
title_list = ['→', '↘', '↓', '↙', '←', '↖', '↑', '↗']

In [ ]:
region_masks = fl.load(path / 'regions_masks.h5')

In [ ]:
regions_list = ['l_habenula_coords',
    'r_habenula_coords',
    'ipn_coords',
    'ahb_coords',
    'pretectum_coords']

In [ ]:
region_names = ['L habenula', 'R habenula', 'IPN', 'aHB', 'Pretectum']

In [ ]:
########## plot the distribution of correlation values per direction 
fig2, ax2 = plt.subplots(5,8, figsize=(12, 6), sharex=True, sharey=True)
plt.subplots_adjust(left=0.05, wspace=0.2, hspace=0.4)

for region in range(len(region_names)):
    
    region_idx = region_masks[regions_list[region]]
    for direction in range(8):
        ax2[region, direction].hist(reg_corr_values[region_idx,direction], bins=100, density=False, color=color_list[direction])
        ax2[region, direction].spines['right'].set_visible(False)
        ax2[region, direction].spines['top'].set_visible(False)
        ax2[region, direction].set_xlim(-1,1)
        
        ax2[0, direction].set_title(title_list[direction])

        if direction > 0:
            ax2[region, direction].spines['left'].set_visible(False)
        else:
            ax2[region, direction].set_ylabel(region_names[region])

In [ ]:
file_name = "corr dist per region.jpg"
fig2.savefig(path / file_name, dpi=300)
file_name = "corr dist per region.pdf"
fig2.savefig(path / file_name, dpi=300)